In [ ]:
import argparse
import multiprocessing
import os

import numpy as np
import pandas as pd
import yaml
from itertools import chain

from config import (SPECS_COMPLETE_YAMLS, HEURISTIC_RESULTS_DIR,
                    SP_RESULTS_DIR, RO_RESULTS_DIR)
from model import DC, LPAC, Solution
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f = 0
    r_hat = 3
    casestudy = 'imelda'
    pftype = 'dc'
    approach = 'stochastic'
    timelimit = 45 * 60
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--f')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    parser.add_argument('--timelimit')
    args = parser.parse_args()
    f = int(args.f)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    timelimit = int(args.timelimit)

In [ ]:
if approach == 'stochastic':
    MY_RESULTS_DIR = SP_RESULTS_DIR
elif approach == 'robust':
    MY_RESULTS_DIR = RO_RESULTS_DIR
else:
    raise ValueError('`approach` must be either "stochastic" or "robust"')

In [ ]:
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = approach
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]
    probability = pd.Series(specs['probability'])

heur_sols_filename = os.path.join(HEURISTIC_RESULTS_DIR, approach, f'solutions-{casestudy}-r{r_hat}.yaml')
with open(heur_sols_filename) as fh:
    sols_data = yaml.load(fh, Loader=yaml.Loader)

perf_data = dict()
for omega in specs['Omega']:
    heur_perf_filename = os.path.join(HEURISTIC_RESULTS_DIR, approach, f'perf-{pftype}-{casestudy}-{omega}-r{r_hat}.yaml')
    with open(heur_perf_filename) as fh:
        perf_data.update(yaml.load(fh, Loader=yaml.Loader))
sr_perf = pd.Series(perf_data)
sr_perf.index.names = ['f', 'weight_load', 'weight_flow', 'omega']
if approach == 'stochastic':
    df_perf = sr_perf.unstack('omega').multiply(probability).sum(axis=1).unstack(['weight_load', 'weight_flow'])
elif approach == 'robust':
    df_perf = sr_perf.unstack('omega').max(axis=1).unstack(['weight_load', 'weight_flow'])
else:
    raise ValueError('`approach` must be either "stochastic" or "robust"')

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# get the appropriate model class
modelcls = model_classes[pftype]

# instantiate a model
MODEL = modelcls(**specs)

# budget constraint
MODEL.con_resource_hi.RHS = f

# load heuristic solution
weight_load, weight_flow = df_perf.loc[f].idxmin()
x_heur = list(chain.from_iterable(sols_data[f, weight_load, weight_flow]))
for (k, r) in MODEL.x:
    MODEL.x[k, r].Start = int((k, r) in x_heur)

# update
MODEL.update()

In [ ]:
# if solving an LPAC variant and DC solution is available, load it
zipfile = os.path.join(MY_RESULTS_DIR, f'{casestudy}-dc-f{f}-r{r_hat}.zip')
if os.path.exists(zipfile):
    MODEL.model.NumStart += 1
    MODEL.model.setParam('StartNumber', MODEL.model.NumStart)
    dc_solution = Solution.from_zip(zipfile)
    for (k, r), indicator in dc_solution['x'].round().items():
        MODEL.x[k, r].Start = int(indicator > 0.5)

In [ ]:
os.makedirs(MY_RESULTS_DIR, exist_ok=True)
logfile = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.log')
zipfile = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')

MODEL.model.setParam('MIPGap', 0.00)
MODEL.model.setParam('TimeLimit', timelimit)
MODEL.model.setParam('LogFile', logfile)
MODEL.model.setParam('Threads', np.floor(multiprocessing.cpu_count()))
MODEL.model.setParam('ImpliedCuts', 2)
MODEL.model.setParam('PreSolve', 2)
MODEL.model.setParam('CutPasses', 10)
MODEL.model.setParam('MIPFocus', 2)
MODEL.update()
MODEL.solve()

sol = Solution.from_solved_instance(MODEL)
sol.to_zip(zipfile, variables=['x', 'ObjBound', 'ObjVal', 'gamma', 'gamma_under', 'gamma_over'])